# Web3 Exploration

Explore Ethereum blockchain data using Web3.py and Infura.

In [20]:
import sys
from pathlib import Path

# Add project root to Python path
PROJECT_ROOT = Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from utils import get_infura_web3

## Connect to Ethereum

Create an HTTP connection to Infura and fetch the latest block:

In [21]:
# Connect to Infura
w3 = get_infura_web3()

print(f"Connected: {w3.is_connected()}")
print(f"Chain ID: {w3.eth.chain_id}")

Connected: True
Chain ID: 1


In [22]:
# Get the latest block
latest_block = w3.eth.get_block('latest')

print(f"Latest Block Number: {latest_block['number']}")
print(f"Block Hash: {latest_block['hash'].hex()}")
print(f"Timestamp: {latest_block['timestamp']}")
print(f"Gas Used: {latest_block['gasUsed']:,}")
print(f"Gas Limit: {latest_block['gasLimit']:,}")
print(f"Transaction Count: {len(latest_block['transactions'])}")

Latest Block Number: 23929408
Block Hash: f7e9a16f8f1d10e4693c930024b76985b0c9968312c8f639c9a45e2a028229e3
Timestamp: 1764722375
Gas Used: 59,566,031
Gas Limit: 59,941,408
Transaction Count: 248


## Gas Price Information

In [23]:
# Get current gas price
gas_price_wei = w3.eth.gas_price
gas_price_gwei = w3.from_wei(gas_price_wei, 'gwei')

print(f"Current Gas Price: {gas_price_gwei:.2f} Gwei")

Current Gas Price: 0.04 Gwei


In [24]:
block = w3.eth.get_block(23928359 - 1)
block

AttributeDict({'baseFeePerGas': 50521529,
 'blobGasUsed': 1048576,
 'difficulty': 0,
 'excessBlobGas': 80609280,
 'extraData': HexBytes('0x546974616e2028746974616e6275696c6465722e78797a29'),
 'gasLimit': 59765917,
 'gasUsed': 16032386,
 'hash': HexBytes('0x180392bb4c49db341d2e773ecb53261718bbd6987bd2d5d988e1cb24b219238c'),
 'logsBloom': HexBytes('0x54ee394e8d75923b340e540db33abf893dd723f64a44a8c33623b0537edd3f51877e051ab2ab0092eaf11f03a973c3ff7e219a679d38a8a586fc23cafd376193b5dce5da03c9bbdaf91b753dd3f3fe36624c6696c55667a35de2dc57ebfcb5481e0cdb4f8e2ee554a43ba5315dd95d12b30aca2f4c3436de6c7cf735197a46557de9bc09daba88f7ca6da620e1964fc2a14fa171d5746cce3b3118d2c239cd5b2ef0472b6d9d3cc53f4175c6dab905f9342677aa39afda792de70abcd39f7ad6c9987e3f57c78a75ad59681a62b0fc8fc6c4625f662417f2f4ec1d5afd94edcb6cb9be700b34dc71b2441618ab46cf87a64bbb089fdd455e79c5a7415028388f'),
 'miner': '0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97',
 'mixHash': HexBytes('0xf1260bd34f830434d30d675661e781b9ea4f474adcb87f49b95d9

In [25]:
transactions = block['transactions']
transaction = transactions[0]
transaction

HexBytes('0x45545d18761620cc1538e83520155426e815f65e5887757c569229a308c87bea')

In [29]:
transaction_dict = w3.eth.get_transaction(transaction)
receipt = w3.eth.get_transaction_receipt(transaction)
# transaction_dict
# receipt

In [30]:
effective_gas_price = receipt["effectiveGasPrice"]  # post-London
base_fee = block["baseFeePerGas"]
priority_fee_per_gas = effective_gas_price - base_fee
gas_used = receipt["gasUsed"]
gas_tip_paid = priority_fee_per_gas * gas_used
gas_tip_paid

0

### Miner Fee Per Block

In [32]:
block_number = 23928350
def get_miner_fee(block_number):
    """Tips to validator on EXECUTION layer. 
    
    DOES NOT INCLUDE:
    1. blob fees (EIP-4844)
    2. Fees paid directly to the miner.
    
    """
    
    block = w3.eth.get_block(block_number, full_transactions=False)
    
    base_fee = block["baseFeePerGas"]
    fee_recipient = block["miner"]  # proposer / validator
    
    total_tip_wei = 0
    
    for tx_hash in block["transactions"]:
        receipt = w3.eth.get_transaction_receipt(tx_hash)
        effective_gas_price = receipt["effectiveGasPrice"]
        gas_used = receipt["gasUsed"]
    
        priority_fee_per_gas = effective_gas_price - base_fee
        if priority_fee_per_gas < 0:
            # Shouldn't happen in a valid block, but just in case
            priority_fee_per_gas = 0
    
        total_tip_wei += priority_fee_per_gas * gas_used
        
    return total_tip_wei


# total_tip_wei = get_miner_fee(block_number)
# print("Tips to validator (execution layer only):", total_tip_wei, "wei")
# print("≈", w3.from_wei(total_tip_wei, "ether"), "ETH")

In [34]:
import time
import random

def retry_with_backoff(func, max_retries=5, base_delay=0.5, max_delay=30):
    """
    Retry a function with exponential backoff.
    
    Args:
        func: Callable to execute
        max_retries: Maximum number of retry attempts
        base_delay: Initial delay in seconds
        max_delay: Maximum delay between retries
        
    Returns:
        Result of the function call
        
    Raises:
        Exception: If all retries are exhausted
    """
    for attempt in range(max_retries):
        try:
            return func()
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            # Exponential backoff with jitter
            delay = min(base_delay * (2 ** attempt) + random.uniform(0, 1), max_delay)
            time.sleep(delay)


def transactions_with_address(block_number, address):
    """Get all transactions in a block that have a specific address as recipient."""
    block = retry_with_backoff(lambda: w3.eth.get_block(block_number))
    txs = []
    transactions = block['transactions']
    for transaction in transactions:
        transaction_dict = retry_with_backoff(lambda tx=transaction: w3.eth.get_transaction(tx))
        to_addr = transaction_dict.get('to')
        if to_addr and address.lower() == to_addr.lower():
            txs.append(transaction)
    return txs

In [36]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def transactions_with_address_parallel(block_numbers, address, max_workers=10):
    """
    Process multiple blocks in parallel to find transactions involving an address.
    
    Args:
        block_numbers: List of block numbers to process
        address: The address to search for
        max_workers: Number of parallel threads (default 10)
        
    Returns:
        List of lists, where each inner list contains transactions for that block
    """
    results = [None] * len(block_numbers)
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks and track which block each future corresponds to
        future_to_idx = {
            executor.submit(transactions_with_address, block_num, address): idx
            for idx, block_num in enumerate(block_numbers)
        }
        
        # Collect results as they complete
        for future in as_completed(future_to_idx):
            idx = future_to_idx[future]
            results[idx] = future.result()
    
    return results


# Example: Process blocks 23928350 to 23928359 in parallel
address = "0x6CA298D2983aB03Aa1dA7679389D955A4eFEE15C"
block_numbers = list(range(23928350, 23928360))

start = time.time()
results = transactions_with_address_parallel(block_numbers, address, max_workers=10)
end = time.time()

print(f"Processed {len(block_numbers)} blocks in {end - start:.2f} seconds")
print(f"Results: {results}")

Processed 10 blocks in 243.59 seconds
Results: [[], [], [], [], [], [], [], [], [], []]
